# Alphavec Example - Simulate

This notebook demonstrates running `alphavec.simulate()` on the bundled feather datasets in `examples/data/`.

The weights were generated by a realworld systematic trading strategy.

Install the optional dependencies to run this notebook: `pip install -r requirements.dev.txt`

In [ ]:
from pathlib import Path

from IPython.display import HTML, display
import pandas as pd

from alphavec import MarketData, SimConfig, simulate, tearsheet

## Load Data

In [ ]:
data_dir = Path("./data")

weights = pd.read_feather(data_dir / "weights.feather")
close_prices = pd.read_feather(data_dir / "close_prices.feather")
open_prices = pd.read_feather(data_dir / "open_prices.feather")
funding_rates = pd.read_feather(data_dir / "funding_rates.feather")

display(weights.tail())
display(close_prices.tail())

symbols = weights.columns.tolist()
print("assets", len(symbols))
print("start", weights.index.min())
print("end", weights.index.max())

## Align Data

In [ ]:
weights = weights.sort_index()

# Simulate requires that all given dataframes have the same index and columns as close_prices
assert weights.index.equals(close_prices.index)
assert weights.columns.equals(close_prices.columns)

# Drop leading periods before the strategy has any active (non-NaN) weights and align all dataframes
active_mask = weights.notna().any(axis=1)
weights = weights.loc[active_mask.idxmax() :]
close_prices = close_prices.loc[weights.index, weights.columns]
open_prices = open_prices.loc[weights.index, weights.columns]
funding_rates = funding_rates.loc[weights.index, weights.columns]

# Use next-period open as the execution price to avoid lookahead.
exec_prices = open_prices.shift(-1)

## Run Simulation

In [ ]:
result = simulate(
    weights=weights,
    market=MarketData(
        close_prices=close_prices,
        exec_prices=exec_prices,
        funding_rates=funding_rates,
    ),
    config=SimConfig(
        benchmark_asset="BTC",
        order_notional_min=10,
        fee_rate=0.00025,      # 2.5 bps per trade
        slippage_rate=0.001,   # 10 bps per trade
        init_cash=10_000,
        freq_rule="1D",
        trading_days_year=365,
        risk_free_rate=0.03,
    ),
)

html_str = tearsheet(sim_result=result, output_path="simulate_tearsheet.html", smooth_periods=30)
display(HTML(html_str))
